# Klasifikasi Gambar Sederhana dengan CIFAR-10

Notebook ini mendemonstrasikan klasifikasi gambar menggunakan:

**Dataset:** [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) — 60.000 gambar berwarna 32x32 pixel, 10 kelas

**Model yang digunakan:**
1. K-Nearest Neighbors (KNN)
2. Support Vector Machine (SVM)
3. Random Forest
4. Convolutional Neural Network (CNN) Sederhana

**10 Kelas:** airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck

## 1. Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from time import time

# Scikit-learn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix
)
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# TensorFlow / Keras
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
)
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

import warnings
warnings.filterwarnings('ignore')

print(f'TensorFlow version: {tf.__version__}')
print('Libraries loaded!')

## 2. Load & Eksplorasi Dataset CIFAR-10

CIFAR-10 tersedia langsung dari `tensorflow.keras.datasets`. Dataset ini terdiri dari:
- **60.000 gambar** berwarna 32×32 pixel
- **10 kelas:** airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck
- **50.000 training** + **10.000 testing**

Sumber: https://www.cs.toronto.edu/~kriz/cifar.html

In [ ]:
# Load dataset
(X_train_full, y_train_full), (X_test, y_test) = cifar10.load_data()

# Flatten labels
y_train_full = y_train_full.flatten()
y_test = y_test.flatten()

# Nama kelas
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

print(f'Training set  : {X_train_full.shape} — {len(y_train_full)} gambar')
print(f'Test set      : {X_test.shape} — {len(y_test)} gambar')
print(f'Ukuran gambar : {X_train_full.shape[1]}x{X_train_full.shape[2]} pixel, {X_train_full.shape[3]} channel (RGB)')
print(f'Jumlah kelas  : {len(class_names)}')
print(f'Kelas         : {class_names}')

In [ ]:
# Visualisasi distribusi kelas
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

for ax, (data, title) in zip(axes, [(y_train_full, 'Training Set'), (y_test, 'Test Set')]):
    unique, counts = np.unique(data, return_counts=True)
    colors = plt.cm.tab10(np.linspace(0, 1, 10))
    ax.bar([class_names[i] for i in unique], counts, color=colors, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Distribusi Kelas — {title}', fontsize=13, fontweight='bold')
    ax.set_ylabel('Jumlah Gambar')
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Tampilkan contoh gambar dari setiap kelas
fig, axes = plt.subplots(2, 5, figsize=(15, 7))

for i, ax in enumerate(axes.flat):
    # Ambil 1 gambar random dari kelas i
    idx = np.where(y_train_full == i)[0]
    random_idx = np.random.choice(idx, 6)
    
    # Buat grid 2x3 dari gambar kelas tersebut
    grid = np.concatenate([
        np.concatenate([X_train_full[random_idx[j]] for j in range(3)], axis=1),
        np.concatenate([X_train_full[random_idx[j]] for j in range(3, 6)], axis=1)
    ], axis=0)
    
    ax.imshow(grid)
    ax.set_title(class_names[i], fontsize=12, fontweight='bold')
    ax.axis('off')

plt.suptitle('Contoh Gambar dari Setiap Kelas CIFAR-10', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()

## 3. Preprocessing Data

Untuk model ML tradisional (KNN, SVM, Random Forest):
- Gunakan subset lebih kecil (5000 train, 1000 test) agar waktu training reasonable
- Flatten gambar 32×32×3 → vektor 3072 dimensi
- Normalisasi ke [0, 1]
- PCA untuk reduksi dimensi

In [ ]:
# === Subset untuk ML Tradisional (agar training tidak terlalu lama) ===
N_TRAIN = 5000
N_TEST = 1000

# Random sampling
np.random.seed(42)
train_idx = np.random.choice(len(X_train_full), N_TRAIN, replace=False)
test_idx = np.random.choice(len(X_test), N_TEST, replace=False)

X_train_ml = X_train_full[train_idx]
y_train_ml = y_train_full[train_idx]
X_test_ml = X_test[test_idx]
y_test_ml = y_test[test_idx]

# Flatten: (N, 32, 32, 3) → (N, 3072)
X_train_flat = X_train_ml.reshape(N_TRAIN, -1).astype('float32') / 255.0
X_test_flat = X_test_ml.reshape(N_TEST, -1).astype('float32') / 255.0

print(f'ML Tradisional — Train: {X_train_flat.shape}, Test: {X_test_flat.shape}')

# Standardisasi
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_flat)
X_test_scaled = scaler.transform(X_test_flat)

# PCA — reduksi dari 3072 ke 100 komponen
N_COMPONENTS = 100
pca = PCA(n_components=N_COMPONENTS, random_state=42)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

explained_var = pca.explained_variance_ratio_.sum() * 100
print(f'PCA: {N_COMPONENTS} komponen menjelaskan {explained_var:.1f}% variance')

In [ ]:
# Visualisasi PCA — 2D projection
pca_2d = PCA(n_components=2, random_state=42)
X_2d = pca_2d.fit_transform(X_train_scaled)

plt.figure(figsize=(10, 8))
colors = plt.cm.tab10(np.linspace(0, 1, 10))

for i in range(10):
    mask = y_train_ml == i
    plt.scatter(X_2d[mask, 0], X_2d[mask, 1], c=[colors[i]], label=class_names[i],
                alpha=0.5, s=15, edgecolors='none')

plt.xlabel('PC1', fontsize=12)
plt.ylabel('PC2', fontsize=12)
plt.title('PCA 2D Projection — CIFAR-10', fontsize=14, fontweight='bold')
plt.legend(fontsize=9, markerscale=2, loc='best')
plt.tight_layout()
plt.show()

## 4. Model 1 — K-Nearest Neighbors (KNN)

In [ ]:
print('Training KNN...')
t0 = time()

knn = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
knn.fit(X_train_pca, y_train_ml)
y_pred_knn = knn.predict(X_test_pca)

acc_knn = accuracy_score(y_test_ml, y_pred_knn)
t_knn = time() - t0

print(f'KNN Accuracy  : {acc_knn:.4f} ({acc_knn*100:.2f}%)')
print(f'Training Time : {t_knn:.2f}s')
print()
print(classification_report(y_test_ml, y_pred_knn, target_names=class_names))

## 5. Model 2 — Support Vector Machine (SVM)

In [ ]:
print('Training SVM (RBF kernel)...')
t0 = time()

svm = SVC(kernel='rbf', C=10, gamma='scale', random_state=42)
svm.fit(X_train_pca, y_train_ml)
y_pred_svm = svm.predict(X_test_pca)

acc_svm = accuracy_score(y_test_ml, y_pred_svm)
t_svm = time() - t0

print(f'SVM Accuracy  : {acc_svm:.4f} ({acc_svm*100:.2f}%)')
print(f'Training Time : {t_svm:.2f}s')
print()
print(classification_report(y_test_ml, y_pred_svm, target_names=class_names))

## 6. Model 3 — Random Forest

In [ ]:
print('Training Random Forest...')
t0 = time()

rf = RandomForestClassifier(n_estimators=300, max_depth=20, random_state=42, n_jobs=-1)
rf.fit(X_train_pca, y_train_ml)
y_pred_rf = rf.predict(X_test_pca)

acc_rf = accuracy_score(y_test_ml, y_pred_rf)
t_rf = time() - t0

print(f'RF Accuracy   : {acc_rf:.4f} ({acc_rf*100:.2f}%)')
print(f'Training Time : {t_rf:.2f}s')
print()
print(classification_report(y_test_ml, y_pred_rf, target_names=class_names))

## 7. Model 4 — CNN Sederhana (Deep Learning)

CNN lebih cocok untuk klasifikasi gambar karena dapat menangkap fitur spasial (tepi, tekstur, pola) yang tidak bisa ditangkap oleh model tradisional.

In [ ]:
# Preprocessing untuk CNN — normalisasi & one-hot encoding
X_train_cnn = X_train_full.astype('float32') / 255.0
X_test_cnn = X_test.astype('float32') / 255.0

y_train_cat = to_categorical(y_train_full, 10)
y_test_cat = to_categorical(y_test, 10)

print(f'CNN Train: {X_train_cnn.shape}, Labels: {y_train_cat.shape}')
print(f'CNN Test : {X_test_cnn.shape}, Labels: {y_test_cat.shape}')

In [ ]:
# Arsitektur CNN Sederhana
model = Sequential([
    # Block 1
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
    BatchNormalization(),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    # Block 2
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    # Block 3
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    # Classifier
    Flatten(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:
# Training CNN
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

print('Training CNN...')
t0 = time()

history = model.fit(
    X_train_cnn, y_train_cat,
    epochs=30,
    batch_size=64,
    validation_split=0.1,
    callbacks=[early_stop],
    verbose=1
)

t_cnn = time() - t0
print(f'\nTraining Time: {t_cnn:.1f}s')

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy
axes[0].plot(history.history['accuracy'], label='Train', linewidth=2)
axes[0].plot(history.history['val_accuracy'], label='Validation', linewidth=2)
axes[0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Loss
axes[1].plot(history.history['loss'], label='Train', linewidth=2)
axes[1].plot(history.history['val_loss'], label='Validation', linewidth=2)
axes[1].set_title('Model Loss', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Evaluasi CNN
loss, acc_cnn = model.evaluate(X_test_cnn, y_test_cat, verbose=0)

y_pred_cnn = model.predict(X_test_cnn, verbose=0)
y_pred_cnn_labels = np.argmax(y_pred_cnn, axis=1)

print(f'CNN Test Accuracy : {acc_cnn:.4f} ({acc_cnn*100:.2f}%)')
print(f'CNN Test Loss     : {loss:.4f}')
print()
print(classification_report(y_test, y_pred_cnn_labels, target_names=class_names))

## 8. Perbandingan Semua Model

In [ ]:
# Ringkasan performa
results = {
    'Model': ['KNN (k=5)', 'SVM (RBF)', 'Random Forest', 'CNN'],
    'Accuracy': [acc_knn, acc_svm, acc_rf, acc_cnn],
    'Training Time (s)': [t_knn, t_svm, t_rf, t_cnn],
    'Data Used': ['5K (PCA)', '5K (PCA)', '5K (PCA)', '50K (Full)']
}

print('=' * 70)
print(f'{"Model":<18} {"Accuracy":>10} {"Time (s)":>12} {"Data":>15}')
print('=' * 70)
for i in range(4):
    print(f'{results["Model"][i]:<18} {results["Accuracy"][i]:>10.4f} {results["Training Time (s)"][i]:>12.2f} {results["Data Used"][i]:>15}')
print('=' * 70)

In [ ]:
# Bar chart perbandingan accuracy
fig, ax = plt.subplots(figsize=(10, 6))

models = results['Model']
accs = results['Accuracy']
colors_bar = ['#065A82', '#1C7293', '#21295C', '#F96167']

bars = ax.bar(models, accs, color=colors_bar, edgecolor='black', linewidth=0.5, width=0.6)

# Label di atas bar
for bar, acc in zip(bars, accs):
    ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.01,
            f'{acc*100:.1f}%', ha='center', fontsize=13, fontweight='bold')

ax.set_ylabel('Accuracy', fontsize=13)
ax.set_title('Perbandingan Accuracy — Semua Model', fontsize=15, fontweight='bold')
ax.set_ylim(0, 1.1)
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 9. Confusion Matrix — CNN (Model Terbaik)

In [ ]:
# Confusion Matrix untuk CNN
cm = confusion_matrix(y_test, y_pred_cnn_labels)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names,
            linewidths=0.5, linecolor='gray')
plt.xlabel('Predicted', fontsize=13)
plt.ylabel('Actual', fontsize=13)
plt.title('Confusion Matrix — CNN pada CIFAR-10', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()

## 10. Visualisasi Prediksi CNN

In [ ]:
# Tampilkan 20 prediksi random
np.random.seed(123)
random_idx = np.random.choice(len(X_test), 20, replace=False)

fig, axes = plt.subplots(4, 5, figsize=(15, 12))

for i, ax in enumerate(axes.flat):
    idx = random_idx[i]
    ax.imshow(X_test[idx])
    
    true_label = class_names[y_test[idx]]
    pred_label = class_names[y_pred_cnn_labels[idx]]
    confidence = np.max(y_pred_cnn[idx]) * 100
    
    correct = true_label == pred_label
    color = 'green' if correct else 'red'
    symbol = '✓' if correct else '✗'
    
    ax.set_title(f'{symbol} {pred_label}\n({confidence:.0f}%)',
                 fontsize=10, color=color, fontweight='bold')
    ax.axis('off')

plt.suptitle('Prediksi CNN — Hijau: Benar, Merah: Salah',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Tampilkan gambar yang SALAH diprediksi (misclassified)
misclassified = np.where(y_pred_cnn_labels != y_test)[0]
print(f'Total misclassified: {len(misclassified)} dari {len(y_test)} ({len(misclassified)/len(y_test)*100:.1f}%)')

# Tampilkan 15 contoh salah prediksi
fig, axes = plt.subplots(3, 5, figsize=(15, 9))
sample_wrong = np.random.choice(misclassified, min(15, len(misclassified)), replace=False)

for i, ax in enumerate(axes.flat):
    if i < len(sample_wrong):
        idx = sample_wrong[i]
        ax.imshow(X_test[idx])
        true_l = class_names[y_test[idx]]
        pred_l = class_names[y_pred_cnn_labels[idx]]
        conf = np.max(y_pred_cnn[idx]) * 100
        ax.set_title(f'True: {true_l}\nPred: {pred_l} ({conf:.0f}%)',
                     fontsize=9, color='red', fontweight='bold')
    ax.axis('off')

plt.suptitle('Contoh Gambar yang Salah Diprediksi oleh CNN',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 11. Per-Class Accuracy Analysis

In [ ]:
# Per-class accuracy dari CNN
per_class_acc = []
for i in range(10):
    mask = y_test == i
    class_acc = accuracy_score(y_test[mask], y_pred_cnn_labels[mask])
    per_class_acc.append(class_acc)

# Sort by accuracy
sorted_idx = np.argsort(per_class_acc)

plt.figure(figsize=(10, 6))
colors_h = ['#F96167' if acc < 0.8 else '#10B981' if acc > 0.9 else '#F59E0B'
             for acc in np.array(per_class_acc)[sorted_idx]]

bars = plt.barh([class_names[i] for i in sorted_idx],
                np.array(per_class_acc)[sorted_idx],
                color=colors_h, edgecolor='black', linewidth=0.5)

for bar, acc in zip(bars, np.array(per_class_acc)[sorted_idx]):
    plt.text(bar.get_width() + 0.005, bar.get_y() + bar.get_height() / 2,
             f'{acc*100:.1f}%', va='center', fontsize=11, fontweight='bold')

plt.xlabel('Accuracy', fontsize=13)
plt.title('Per-Class Accuracy — CNN pada CIFAR-10', fontsize=14, fontweight='bold')
plt.xlim(0, 1.1)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print('\nKelas dengan accuracy terendah cenderung mirip secara visual (misal: cat vs dog, automobile vs truck)')

## 12. Kesimpulan

### Hasil Perbandingan Model

| Model | Accuracy | Catatan |
|-------|----------|------|
| KNN | ~35-40% | Sangat sederhana, tidak cocok untuk gambar kompleks |
| SVM | ~45-50% | Lebih baik dari KNN, tapi terbatas pada fitur flat |
| Random Forest | ~35-40% | Mirip KNN, kurang optimal untuk data gambar |
| **CNN** | **~80-85%** | **Jauh lebih unggul**, mampu menangkap fitur spasial |

### Insight
1. **CNN jauh lebih unggul** untuk klasifikasi gambar karena menggunakan konvolusi untuk mendeteksi pola visual
2. **Model ML tradisional** terbatas karena menggunakan pixel sebagai fitur tanpa memahami struktur spasial
3. **PCA membantu** model tradisional dengan mereduksi dimensi, tapi tetap kehilangan informasi spasial
4. **Kelas yang mirip secara visual** (cat vs dog, automobile vs truck) lebih sulit diklasifikasi

### Dataset Info
- **Sumber:** [CIFAR-10 — Canadian Institute For Advanced Research](https://www.cs.toronto.edu/~kriz/cifar.html)
- **Download:** Otomatis melalui `tensorflow.keras.datasets.cifar10`
- **Lisensi:** Untuk penggunaan akademik dan penelitian